In [2]:
import numpy as np
import cv2

# загрузите изображение, смените цвет на оттенки серого и уменьшите резкость
image = cv2.imread("images/series1/1_1.bmp")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
cv2.imwrite("result/1_1_gray.bmp", gray)

True

In [18]:
# параметры цветового фильтра
hsv_min = np.array((7, 40, 60), np.uint8)
hsv_max = np.array((15, 255, 200), np.uint8)

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # меняем цветовую модель с BGR на HSV 
cv2.imwrite("result/1_1_hsv.bmp", hsv)
cont = cv2.inRange(hsv, hsv_min, hsv_max)
cv2.imwrite("result/1_1_cont.bmp", cont)

True

In [19]:
seed_pt = (25, 25)
fill_color = 0
mask = np.zeros_like(cont)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
for th in range(60, 120):
    prev_mask = mask.copy()
    mask = cv2.threshold(cont, th, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.floodFill(mask, None, seed_pt, fill_color)[1]
    mask = cv2.bitwise_or(mask, prev_mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

n_centers = cv2.connectedComponents(mask)[0] - 1
print('There are %d cells in the image.'%n_centers)

There are 545 cells in the image.
